In [1]:
# Homework 1

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Jesse D. Jenkins and Michael R. Davidson (last updated: September 14, 2022)_

This Notebook will walk you through defining a simple transport flow model and then ask you to interact with the solutions and modify to model to add additional constraints...

In [2]:
using JuMP
using HiGHS
using DataFrames
using CSV

### Define sets

We will define two sets, both as arrays of strings

***Production plants, $P$***

In [3]:
P=["trenton", "newark"] # production plants

2-element Vector{String}:
 "trenton"
 "newark"

***Markets for products, $M$***

In [4]:
M=["newyork", "princeton", "philadelphia"] # markets for products

3-element Vector{String}:
 "newyork"
 "princeton"
 "philadelphia"

Note that sets can also be defined over intervals (as in `i=1:10`) or numerical vectors (as in `x=[2, 4, 5, 11]`) 

### Define parameters

We'll make use of the defined sets as indexes for our parameters...

***Plant production capacities***

In [5]:
plants = DataFrame(plant=P, capacity=[350,650])

Row,plant,capacity
,String,Int64
1,trenton,350
2,newark,650


***Demand for products***

Stored in a [DataFrame](https://juliadata.github.io/DataFrames.jl/stable/)

In [6]:
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275]
)

Row,market,demand
,String,Int64
1,newyork,325
2,princeton,300
3,philadelphia,275


A few different ways to index into our DataFrames to access parameters (all of the below are equivalent)

In [7]:
plants[plants.plant.=="newark",:capacity] # option 1

1-element Vector{Int64}:
 650

In [8]:
plants[plants.plant.=="newark",:].capacity # option 2

1-element Vector{Int64}:
 650

In [9]:
plants.capacity[plants.plant.=="newark"] # option 3

1-element Vector{Int64}:
 650

In [10]:
plants[:,:capacity][plants.plant.=="newark"] # option 4

1-element Vector{Int64}:
 650

Note that DataFrame indexing returns an Array by default, in this case, a 1-element Array of type Int64 (64-bit integer), as indicated by `Array{Int64,1}` above. 

To access the single Int64 value, append `[1]` to any of the above to reference the first (and only) element in this array. 

In [11]:
plants.capacity[plants.plant.=="newark"][1]

650

In [12]:
typeof(plants.capacity[plants.plant.=="newark"][1])

Int64

In [13]:
typeof(plants.capacity[plants.plant.=="newark"])

Vector{Int64} (alias for Array{Int64, 1})

***Distance from plants to markets***

Stored in a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/v0.19/containers/) with data array and symbolic references across each of our sets (plants and markets), converted to Symbols for referencing

In [14]:
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5;
     0.5 1.5 3.5],
    Symbol.(P),
    Symbol.(M),
)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:trenton, :newark]
    Dimension 2, [:newyork, :princeton, :philadelphia]
And data, a 2×3 Matrix{Float64}:
 2.5  0.5  1.5
 0.5  1.5  3.5

A couple example references to our DenseAxisArray to access parameters...

In [15]:
distances[:trenton, :newyork] #example of distance references

2.5

In [16]:
distances[:newark, :newyork] #example of distance references

0.5

In [17]:
distances[Symbol(P[2]),Symbol(M[1])] # another way to find distance from newark to trenton

0.5

In [18]:
distances[Symbol("newark"), Symbol("newyork")] # and a third...

0.5

***Costs of transport***

In [19]:
freight_cost = 90 # Cost of freight shipment per unit of distance

90

### Create model
(and specify the HiGHS solver)

In [20]:
transport = Model(HiGHS.Optimizer);

### Define variables

***Quantities of product to transport from plant $p \in P$ to market $m \in M$***

In [21]:
@variable(transport, X[P,M] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Matrix{VariableRef}:
 X[trenton,newyork]  X[trenton,princeton]  X[trenton,philadelphia]
 X[newark,newyork]   X[newark,princeton]   X[newark,philadelphia]

Example reference to single quantity decision variable, the quantity shipped from Newark to Philadelphia:

In [22]:
X["newark","philadelphia"]

X[newark,philadelphia]

### Define constraints

***Supply capacity constraint***

In [23]:
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["trenton", "newark"]
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cSupply[trenton] : X[trenton,newyork] + X[trenton,princeton] + X[trenton,philadelphia] <= 350
 cSupply[newark] : X[newark,newyork] + X[newark,princeton] + X[newark,philadelphia] <= 650

***Demand balance constraint***

Ensure all demand is satisfied at each market

In [24]:
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["newyork", "princeton", "philadelphia"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cDemand[newyork] : X[trenton,newyork] + X[newark,newyork] >= 325
 cDemand[princeton] : X[trenton,princeton] + X[newark,princeton] >= 300
 cDemand[philadelphia] : X[trenton,philadelphia] + X[newark,philadelphia] >= 275

### Define objective function

Minimize total cost of transport to satisfy all demand.

First we'll define an expression for total cost of shipments...

In [25]:
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

Now we'll minimize this total cost

In [26]:
@objective(transport, Min, eCost)

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

## Interact with the model

**(a)** Now let's solve the model. In the blank cell below, enter the command for JuMP to solve a model and run the cell

In [27]:
optimize!(transport)



Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
5 rows, 6 cols, 12 nonzeros
5 rows, 6 cols, 12 nonzeros
Presolve : Reductions: rows 5(-0); columns 6(-0); elements 12(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(900) 0s
          4     8.5500000000e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  8.5500000000e+04
HiGHS run time      :          0.00


**(b)** You've got a solution. Now query the objective function in the empty cell below and save it to a variable (name of your choice)

In [28]:
total_cost = objective_value(transport)


85500.0

**(c)** Now query and save the optimal solution for X (the decisions about shipment quantities from plant to market) to an Array or DataFrame

In [29]:


shipment_df = DataFrame(
    plant = [p for p in P for m in M],
    market = [m for p in P for m in M],
    quantity = [value(X[p, m]) for p in P for m in M]
)


Row,plant,market,quantity
,String,String,Float64
1,trenton,newyork,0.0
2,trenton,princeton,75.0
3,trenton,philadelphia,275.0
4,newark,newyork,325.0
5,newark,princeton,225.0
6,newark,philadelphia,0.0


**(d)** Please interpret your results by writing an explanation in the markdown cell below. 

Which facility or facilities supplies the most demand in New York? Does this result make sense? Why?

Which facility or facilities supplies the most demand in Philadelphia? Does this result make sense? Why?

Which facility or facilities supplies the demand in Princeton? Does this result make sense? Why?

### Analysis of the Transportation Model

---

**Which facility or facilities supply the most demand in New York?wer:} The facility supplying the most demand in New York is \textbf{Newark} with a total of \(325.0\) units. Trenton does not supply any units to New York.

---

**Which facility or facilities supply the most demand in Philadelphia?**

\textbf{Answer:} The facility supplying the most demand in Philadelphia is \textbf{Trenton}, delivering all \(275.0\) units of its demand. Newark does not supply to Philadelphia.

---

**Which facility or facilities supply the demand in Princeton?**

\textbf{Answer:} Both \textbf{Trenton} and \textbf{Newark} service Princeton. Newark takes on a more significant share, providing \(225.0\) units, while Trenton contributes \(75.0\) units.

---

its.


**(e)** A new market in New Brunswick appears, with a demand for 50 units. It is located 1.0 units away from both plants. Add this market to the model and solve again.

In [30]:
# Add "newbrunswick" to your market list
push!(M, "newbrunswick")

# Add demand for "newbrunswick" to the `markets` DataFrame
push!(markets, ["newbrunswick", 50])


# Extend the distances matrix with the distances to "newbrunswick"
new_distances_data = hcat(distances.data, [1.0, 1.0]) # Adding distance of 1.0 for both plants

# Update the distances DenseAxisArray
distances = JuMP.Containers.DenseAxisArray(
    new_distances_data,
    Symbol.(P),
    Symbol.(M)
)


2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:trenton, :newark]
    Dimension 2, [:newyork, :princeton, :philadelphia, :newbrunswick]
And data, a 2×4 Matrix{Float64}:
 2.5  0.5  1.5  1.0
 0.5  1.5  3.5  1.0

**(f)** What is new optimal solution? 

In [31]:
# Redefine the model
transport = Model(HiGHS.Optimizer)

# Redefine the decision variable to incorporate the new market
@variable(transport, X_new[P, M] >= 0)

# Define the new constraints with the expanded variable X_new
@constraint(transport, cSupply[p in P], 
    sum(X_new[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1]
)

@constraint(transport, cDemand[m in M], 
    sum(X_new[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1]
)

@expression(transport, eCost_new, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X_new[p,m] 
        for p in P, m in M)
)
@objective(transport, Min, eCost_new)

# Run the optimization
optimize!(transport)
new_total_cost = objective_value(transport)

# Extract the updated decision values
shipment_df_new = DataFrame(
    plant = [p for p in P for m in M],
    market = [m for p in P for m in M],
    quantity = [value(X_new[p, m]) for p in P for m in M]
)

shipment_df_new


Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
6 rows, 8 cols, 16 nonzeros
6 rows, 8 cols, 16 nonzeros
Presolve : Reductions: rows 6(-0); columns 8(-0); elements 16(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(950) 0s
          5     9.0000000000e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  9.0000000000e+04
HiGHS run time      :          0.00


Row,plant,market,quantity
,String,String,Float64
1,trenton,newyork,0.0
2,trenton,princeton,75.0
3,trenton,philadelphia,275.0
4,trenton,newbrunswick,0.0
5,newark,newyork,325.0
6,newark,princeton,225.0
7,newark,philadelphia,0.0
8,newark,newbrunswick,50.0


**(g)** Interpret this result in the markdown cell below. Which facility or facilities supplies the demand in New Brunswick? Does this result make sense? Why?